In [3]:
from bqplot import pyplot as plt
import bqplot as bq
import ipywidgets as widgets
from src.imports import read_eye_data
from src.analysis import get_sac_onsets_offsets
from src.analysis import predict_possible_saccades
import src.plots as plots
import matplotlib
import numpy as np

# init some values
iTrial   = 0
curr_sac = 0

# get the data
x_matrix,y_matrix  = read_eye_data()
ntrials, trialtime = np.shape(x_matrix)

# predict possible saccades based on old weights
total_predictions  = predict_possible_saccades(x_matrix,y_matrix,sampfreq=1000,min_sacc_dur=12,min_sacc_dist = 10)

# get the predicted saccade onsets and offsets
onsets, offsets    = get_sac_onsets_offsets(total_predictions)
this_trial_n_sacs  = len(onsets[iTrial][0])

# set textbox that shows current trial
current_trial_text = widgets.Text(value="{}".format(iTrial + 1),placeholder='Type something',description='Current trial:',disabled=False)


def next_trial(obj):
    global iTrial
    iTrial += 1
    new_trial()
    update()
    
def previous_trial(obj):
    global iTrial
    iTrial -= 1
    new_trial()
    update()
    
def next_sac(obj):
    global iTrial, this_trial_n_sac, curr_sac
    if curr_sac < this_trial_n_sacs-1:
        curr_sac += 1
    update()

def previous_sac(obj):
    global iTrial, curr_sac
    if curr_sac > 0:
        curr_sac -= 1
    update()

def del_sac(obj):
    global iTrial, curr_sac, this_trial_onsets, this_trial_offsets, this_trial_n_sacs
    this_trial_onsets   = np.delete(this_trial_onsets,curr_sac)
    this_trial_offsets  = np.delete(this_trial_offsets,curr_sac)
    this_trial_n_sacs   = this_trial_n_sacs - 1
    update_patches()
    
    
def update_patches():
    global iTrial, this_trial_onsets, this_trial_offsets, curr_sac , this_trial_n_sacs, patch_x_coords, patch_y_coords
    patch_x_coords = []
    patch_y_coords = []
    
    for sac in range(len(this_trial_onsets)):
        l1= [this_trial_onsets[sac],this_trial_onsets[sac],this_trial_offsets[sac],this_trial_offsets[sac]]
        patch_x_coords.append(l1)
        patch_y_coords.append([-1,1,1,-1])
    
    patch.x = patch_x_coords
    patch.y = patch_y_coords
    

        
    if curr_sac == len(this_trial_onsets):
        patch_curr_sac.x = patch_x_coords[curr_sac-1]
        patch_curr_sac.y = patch_y_coords[curr_sac-1]
    else:
        patch_curr_sac.x = patch_x_coords[curr_sac]
        patch_curr_sac.y = patch_y_coords[curr_sac]
    
    

def new_trial():
    global patch_x_coords, patch_y_coords, this_trial_onsets, this_trial_offsets, this_trial_n_sacs
    # update the patches for this trial
    curr_sac = 0
    this_trial_onsets  = onsets[iTrial][0]
    this_trial_offsets = offsets[iTrial][0]
    this_trial_n_sacs  = len(onsets[iTrial][0])

    patch_x_coords     = []
    patch_y_coords     = []

    for sac in range(len(this_trial_onsets)):
        l1= [this_trial_onsets[sac],this_trial_onsets[sac],this_trial_offsets[sac],this_trial_offsets[sac]]
        patch_x_coords.append(l1)
        patch_y_coords.append([-1,1,1,-1])
    patch.x = patch_x_coords
    patch.y = patch_y_coords
    
    if curr_sac is not None:
        patch_curr_sac.x = patch_x_coords[curr_sac]
        patch_curr_sac.y = patch_y_coords[curr_sac]
        intsel.selected = [patch_x_coords[curr_sac][0]-10,patch_x_coords[curr_sac][2]+10]
    
    
def update():
    global iTrial, this_trial_onsets, this_trial_offsets
    # update the lare line plots
    Line_eye_x.y =  x_matrix[iTrial,:] - np.mean(x_matrix[ iTrial, :])
    Line_eye_y.y =  y_matrix[iTrial,:] - np.mean(y_matrix[ iTrial, :])
    current_trial_text.value =  "{}".format(iTrial + 1) # this gives the current trial that we are working on
    
    # update the current saccade patch
    if curr_sac is not None:
        patch_curr_sac.x = patch_x_coords[curr_sac]
        patch_curr_sac.y = patch_y_coords[curr_sac]
    intsel.selected = [patch_x_coords[curr_sac][0]-10,patch_x_coords[curr_sac][2]+10]    
    update_trace_zoom()
    
def update_trace_zoom(*ignore):
    global iTrial, this_trial_n_sacs, xmin, xmax
    this_trial_n_sacs  = len(onsets[iTrial][0])
    if intsel.selected is not None and len(intsel.selected) == 2:
        xmin, xmax = intsel.selected
        xmin = int(np.round(xmin))
        xmax = int(np.round(xmax))

        # update the plots
        Line_zoom_x.y  = x_matrix[iTrial, xmin : xmax] - np.mean(x_matrix[iTrial, xmin : xmax])
        Line_zoom_y.y  = y_matrix[iTrial, xmin : xmax] - np.mean(y_matrix[iTrial, xmin : xmax])
        Line_zoom_x.x  = np.linspace(xmin,xmax,xmax-xmin)
        Line_zoom_y.x  = np.linspace(xmin,xmax,xmax-xmin)
        x_sc_zoom.min  = xmin
        x_sc_zoom.max  = xmax
        

def edit_sac(*ignore):
    global iTrial, this_trial_onsets, this_trial_offsets, curr_sac, xmin, xmax, new_on

        
    if zoom_selector.selected is not None and len(zoom_selector.selected) == 2:
        new_on, new_off = zoom_selector.selected

        this_trial_onsets[curr_sac]  = round(new_on)
        this_trial_offsets[curr_sac] = round(new_off)
        
        update_patches()
        
def save_trial(*ignore):
    # create again a binary representation of the events
    vec = np.zeros((1000,1))
    for sac in range(len(this_trial_onsets)):
        vec[this_trial_onsets[sac]:this_trial_offsets[sac]:] = np.ones(( this_trial_offsets[sac]-this_trial_onsets[sac],1 ))
    

        
##################################################
#### define buttons and assign their callback ####
##################################################

# trial buttons
next_trial_button = widgets.Button(description='Next trial',disabled=False,button_style='', tooltip='Click me',icon='badger-honey') 
next_trial_button.on_click(next_trial)
previous_trial_button = widgets.Button(description='Previous trial',disabled=False,button_style='',tooltip='Click me', icon='badger-honey') 
previous_trial_button.on_click(previous_trial)

# saccade buttons
next_saccade_button = widgets.Button(description='Next saccade',disabled=False,button_style='', tooltip='Click me',icon='badger-honey') 
next_saccade_button.on_click(next_sac)
previous_saccade_button = widgets.Button(description='Previous saccade',disabled=False,button_style='',tooltip='Click me', icon='badger-honey') 
previous_saccade_button.on_click(previous_sac)

# edit saccades buttons
delete_curr_sac = widgets.Button(description='Delete sac',disabled=False,button_style='', tooltip='Click me',icon='badger-honey') 
delete_curr_sac.on_click(del_sac)

save_trial_button = widgets.Button(description='Save trial',disabled=False,button_style='', tooltip='Click me',icon='badger-honey') 
save_trial_button.on_click(save_trial) 

###################################
### PLOTTING STUFF HAPPENS HERE ###
###################################
animation_time = 300 # this is animation speed

# define axes for different plots
y_ax        = bq.Axis(label="x/y position (dva)",scale=bq.LinearScale(),orientation="vertical")
x_ax        = bq.Axis(label="time (ms)",scale=bq.LinearScale(min=1,max=trialtime),orientation="horizontal")
x_sc_zoom   = bq.LinearScale(min=0)
y_sc_zoom   = bq.LinearScale()
x_zoom_ax   = bq.Axis(label="time (ms)",scale=x_sc_zoom,orientation="horizontal")
y_zoom_ax   = bq.Axis(label="prediction",scale=y_sc_zoom,orientation="vertical")
Line_eye_x  = plt.plot(x=np.arange(trialtime),scales={'x':bq.LinearScale(min=1,max=trialtime),'y':bq.LinearScale()},colors = ['purple'])
Line_eye_y  = plt.plot(x=np.arange(trialtime),scales={'x':bq.LinearScale(min=1,max=trialtime),'y':bq.LinearScale()},colors = ['green'])
Line_zoom_x = plt.plot(scales={'x':x_sc_zoom,'y':y_sc_zoom},colors = ['purple'])
Line_zoom_y = plt.plot(scales={'x':x_sc_zoom,'y':y_sc_zoom},colors = ['green'])


# add a selector for the zoom trace
intsel = bq.interacts.FastIntervalSelector(scale=bq.LinearScale(min=1,max=trialtime),marks=[Line_eye_x],color='red')
intsel.observe(update_trace_zoom,names=['selected'])

# selector within the zoomed view that selects the saccade onsets and offset
new_scale = bq.LinearScale()
zoom_selector = bq.interacts.BrushIntervalSelector(scale=x_sc_zoom,marks=[Line_zoom_x],color='pink')
zoom_selector.observe(edit_sac,names=['selected'])


##############################
### Boxes are defined here ###
##############################

# patch plot that highlights all detected/suggested the saccades
patch = plt.plot(scales={'x':bq.LinearScale(min=1,max=trialtime),'y':bq.LinearScale()},close_path=True,
                 stroke_width=0,fill='inside',opacities=[0.15],colors=['white'])

# patch plot that highlights the current saccade
patch_curr_sac = plt.plot(scales={'x':bq.LinearScale(min=1,max=trialtime),'y':bq.LinearScale()},close_path=True, stroke_width=0,fill='inside',opacities=[0.5])

# patch of current saccade in the zoomed in plot
patch_curr_sac_zoomed = plt.plot(scales={'x':bq.LinearScale(min=1,max=trialtime),'y':bq.LinearScale()},close_path=True, stroke_width=0,fill='inside',opacities=[0.5],color=['white'])

# button box
button_box = widgets.HBox(children= [previous_trial_button,next_trial_button,current_trial_text,previous_saccade_button,next_saccade_button,delete_curr_sac,save_trial_button],
                          layout=widgets.Layout(border=" solid 1px gray",width="1100px"))

# first box on top with eye traces left and zoomed traces right
eyetrace_fig = bq.Figure(animation_duration=animation_time,layout=widgets.Layout(flex='1 1 30%', width='auto'),axes=[x_ax,y_ax],
    marks=[Line_eye_x,Line_eye_y,patch,patch_curr_sac],fig_margin=dict(top=25,bottom=50,left=50,right=25),interaction=intsel)
zoom_fig = bq.Figure(animation_duration=animation_time,layout=widgets.Layout(flex='1 1 0%', width='auto'),axes=[x_zoom_ax,y_zoom_ax],
    marks=[Line_zoom_x,Line_zoom_y],fig_margin=dict(top=25,bottom=50,left=25,right=25),interaction=zoom_selector)

box_layout = widgets.Layout(animation_duration=animation_time,display='flex',flex_flow='row',align_items='stretch',width='100%')
box_0 = widgets.Box(children=[eyetrace_fig, zoom_fig], layout=box_layout)
eyetrace_zoom_box = widgets.VBox([box_0])

# build app layout
app = widgets.VBox(children=[button_box,eyetrace_zoom_box],layout=widgets.Layout(border="solid 2px gray",width="1100px"))
update_trace_zoom()
new_trial()
update()
app

In [17]:
print(len(this_trial_onsets))
print(curr_sac)

4
4
